In [1]:
import json
import numpy as np
import random
import sys

# instance
with open('data/W1-01.json') as f:
    data = json.load(f)

# given solutions
with open('solutions/sol-W1-01.json') as f:
    solution = json.load(f)


In [67]:
nurse_dict = dict()
ALL_NURSES_SET = set() # set of all nurse IDs

# hyperparameters
OVERTIME_P = 1.0  # willingness of nurses to work overtime

class Nurse():
    def __init__(self, id, index, assignments, contract_id, shiftoffrequests):
        self.id = id      # ex: Nurse_1
        self.index = index       # ex: 0 (associated with place in JSON lists)
        self.assignments = assignments # list of shifts
        
        self.overtime_shifts = 0 # nobody has overtime for now
        self.overtime_willingness = 1 # always accept overtime
        
        # contract details
        self.contract_id = contract_id # either Contract_1 or Contract_2
        self.contract_details = self.parse_contract()
        self.max_assignments = self.contract_details["MaximumNumberOfAssignments"]
        
        # assignments
        self.assignments_list = assignments
        self.num_assignments = len([x for x in self.assignments_list if x not in ["R","A","S"]])
        self.remaining_available_days = self.max_assignments - self.num_assignments
        
        # shiftoffrequests
        self.shiftoffrequests = shiftoffrequests
        
    
    def parse_contract(self):
        contract_details = {}
        if self.contract_id == "Contract_1":
            contract_details = data["Contracts"][0]
        elif self.contract_id == "Contract_2":
            contract_details = data["Contracts"][1]
        else:
            print("Invalid contract id.")
            sys.exit(1)
        return contract_details
    
    def sample_overtime(self):
        # CHANGE TO DISTRIBUTION LATER
        return self.overtime_willingness
    
    def list_assignments(self):
        return [day["shift"] for day in solution["Solution"][self.index]["Assignments"]]
    
    def list_shiftoffrequests(self):
        return [day["shift"] for day in data["Solution"][self.index]["Assignments"]]

In [73]:
# collect all shiftoffrequests as dictionary: {Nurse_1: 5,Night}
shiftoffrequests = dict()
for request in data["Shiftoffrequests"]:
    if request["id"] not in shiftoffrequests:
        shiftoffrequests[request["id"]] = list()
    shiftoffrequests[request["id"]].append({request["day"]:request["shift"]})

# print(shiftoffrequests)
    
# make nurse objects, stores shifts in list, indexed by day
index = 0;
for nurse_sol in solution["Solution"]:    
    nurse_id = nurse_sol["id"]
    assignments = [day["shift"] for day in nurse_sol["Assignments"]]
    contract_id = data["Nurses"][day]["contract_id"]
    if nurse_id in shiftoffrequests:
        nurse_requests = shiftoffrequests[nurse_id]
    else: nurse_requests = []
    
    new_nurse = Nurse(nurse_id, index, assignments,contract_id, nurse_requests)
    nurse_dict[nurse_id] = new_nurse
    
    ALL_NURSES_SET.add(nurse_id) # populate set
    
    index+=1
    
    # print(nurse_id)
    # print(assignments)
    # print(contract_id)
    # print(nurse_requests)

In [74]:
NUM_DAYS = 30 # FIND MORE ROBUST WAY 

# rewrite schedule 0-indexed by day
# each index is simply a set of IDs of nurses staffed that day
day_schedule = [set([nurse.id for nurse in nurses 
                 if nurse.assignments[day] not in ["R", "A","S"]]) 
                for day in range(NUM_DAYS)]

print([len(day) for day in day_schedule])

[33, 31, 34, 29, 31, 29, 28, 30, 32, 29, 27, 27, 30, 26, 29, 29, 31, 27, 29, 27, 32, 28, 30, 29, 23, 32, 32, 29, 33, 33]


In [78]:
# list of sets of available nurses per day
available_nurses = []
for day in range(NUM_DAYS):
    print("day ",day)
    staffed_set = day_schedule[day]
    unstaffed_set = ALL_NURSES_SET.difference(staffed_set)
    # print(unstaffed_set)
    
    # check unstaffed nurses for how many total assignments
    available_set = set()
    for unstaffed_nurse in unstaffed_set:
        # print(nurse_dict[unstaffed_nurse].id)
        # print(nurse_dict[unstaffed_nurse].assignments)
        # print(nurse_dict[unstaffed_nurse].contract_id)
        # print(nurse_dict[unstaffed_nurse].max_assignments)
        # print(nurse_dict[unstaffed_nurse].num_assignments)
        # print(nurse_dict[unstaffed_nurse].remaining_available_days)
        # print(nurse_dict[unstaffed_nurse].shiftoffrequests)
        
        if (nurse_dict[unstaffed_nurse].remaining_available_days > 0 
            and str(day+1) not in nurse_dict[unstaffed_nurse].shiftoffrequests
           and nurse_dict[unstaffed_nurse].assignments[day] == "S"):
                available_set.add(unstaffed_nurse)
    
    # WHAT DOES "R" MEAN. CAN IT BE BREACHED. HOW TO TELL IF NIGHT SHIFT/VIOLATION.
    # ONLY INCLUDE "S" (standby??) in available set??
    print(available_set)
    available_nurses.append(available_set)
    
print([len(day) for day in available_nurses])

day  0
set()
day  1
set()
day  2
set()
day  3
{'Nurse_30', 'Nurse_43', 'Nurse_19'}
day  4
{'Nurse_20', 'Nurse_22', 'Nurse_35', 'Nurse_26', 'Nurse_42'}
day  5
{'Nurse_47', 'Nurse_32', 'Nurse_1', 'Nurse_29', 'Nurse_25'}
day  6
{'Nurse_27', 'Nurse_40', 'Nurse_14', 'Nurse_16', 'Nurse_28'}
day  7
{'Nurse_10', 'Nurse_18', 'Nurse_48'}
day  8
{'Nurse_15', 'Nurse_17', 'Nurse_24', 'Nurse_19'}
day  9
{'Nurse_20', 'Nurse_43', 'Nurse_30', 'Nurse_31', 'Nurse_26'}
day  10
{'Nurse_47', 'Nurse_1', 'Nurse_9', 'Nurse_35', 'Nurse_25'}
day  11
{'Nurse_32', 'Nurse_21', 'Nurse_29', 'Nurse_3', 'Nurse_42'}
day  12
{'Nurse_18', 'Nurse_40'}
day  13
{'Nurse_22', 'Nurse_10', 'Nurse_17', 'Nurse_19', 'Nurse_45'}
day  14
{'Nurse_27', 'Nurse_7', 'Nurse_31', 'Nurse_15', 'Nurse_48'}
day  15
{'Nurse_14', 'Nurse_37', 'Nurse_35', 'Nurse_25', 'Nurse_26'}
day  16
{'Nurse_44', 'Nurse_1', 'Nurse_21', 'Nurse_9', 'Nurse_28'}
day  17
{'Nurse_30', 'Nurse_20', 'Nurse_32', 'Nurse_43'}
day  18
{'Nurse_40', 'Nurse_18', 'Nurse_29', 'Nu